# Tensorflow构建cnn模型实现情感分析
本文使用tensorflow框架构建cnn模型，实现口碑数据的情感分析.模型的主要信息如下：
- 输入数据:使用` gensim.models.doc2vec.Doc2Vec(dm=0,dbow_words=1,size=80,min_count=2,iter=55,workers=cores,window=8)`构建PV_DBOW_model模型生成的文本向量
- 模型效果：效果不佳，分类正确率仅0.6左右
- 效果不佳原因：输入数据是80维的doc2vec PV_DBOW_model生成的文本向量，PV_DBOW_mode self-similarity评估效果0.756左右（similar doc per model>0.9句向量表示效果好），故该文本向量表示文本的效果不太好。基于不太理想的数据构建的cnn模型，分类的效果也不会太好

## 读入训练/测试数据

In [1]:
import pandas as pd
pd_train = pd.read_csv('./data/pd_train.csv')
pd_test = pd.read_csv('./data/pd_test.csv')

In [2]:
feature = [str(i) for i in range(80)]

In [3]:
pd_train[feature].as_matrix().shape

(75000, 80)

In [4]:
pd_train['score'].as_matrix().dtype

dtype('int64')

In [5]:
#数据格式为np.array
train_data = pd_train[feature].as_matrix()
#修改数值类型从float64改成float32
train_data = train_data.astype('float32')
train_labels =pd_train['score'].as_matrix()
eval_data = pd_test[feature].as_matrix()
#修改数值类型从float64改成float32
eval_data = eval_data.astype('float32')
eval_labels = pd_test['score'].as_matrix()

In [6]:
train_data.dtype

dtype('float32')

## 构建cnn模型

In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [8]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 80, 1, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=4,
        kernel_size=[10, 1],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 1], strides=2)
    '''
    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=8,
        kernel_size=[2, 1],
        padding="same",
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 1], strides=2)
 '''
    # Dense Layer
    pool2_flat = tf.reshape(pool1, [-1, 40* 1 * 4])
    dense = tf.layers.dense(inputs=pool2_flat, units=32, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout, units=6)

    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [9]:
mnist_classifier = tf.estimator.Estimator(
    model_fn = cnn_model_fn,model_dir='./tmp/pingfen_convnet_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f69e465c290>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': './tmp/pingfen_convnet_model', '_save_summary_steps': 100}


In [10]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

## 训练cnn模型

In [11]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=500,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./tmp/pingfen_convnet_model/model.ckpt-20000
INFO:tensorflow:Saving checkpoints for 20001 into ./tmp/pingfen_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.00014775 0.00118833 0.00090027 0.04385887 0.24246219 0.7114426 ]
 [0.00000043 0.00023146 0.00167531 0.07054105 0.32724586 0.60030586]
 [0.00115522 0.02764106 0.05995301 0.14934221 0.26229778 0.49961066]
 ...
 [0.00043283 0.00752753 0.00438319 0.07313761 0.25316143 0.66135746]
 [0.00000171 0.00014193 0.00555609 0.13779409 0.37296593 0.48354018]
 [0.02396978 0.06482391 0.09996277 0.14773428 0.21547285 0.44803646]]
INFO:tensorflow:loss = 0.9304232, step = 20001
INFO:tensorflow:probabilities = [[0.00043903 0.00757462 0.03466894 0.15001269 0.2830425  0.52426225]
 [0.00005992 0.00233411 0.01474791 0.12201291 0.30385375 0.55699134]
 [0.00020909 0.00265054 0.00719917 0.08118352 0.2497919  0.65896577]
 ...
 [0.00000004 0.0000757  0.00156643 0

INFO:tensorflow:global_step/sec: 51.6978
INFO:tensorflow:probabilities = [[0.00000012 0.00000739 0.00034966 0.0381348  0.2700984  0.69140965]
 [0.01019657 0.05237481 0.02318002 0.11418241 0.23007163 0.56999457]
 [0.00000007 0.00120044 0.00468255 0.15211934 0.39101154 0.45098603]
 ...
 [0.         0.00000645 0.00012105 0.07821114 0.6596096  0.26205173]
 [0.00098777 0.0192467  0.03672789 0.14301829 0.26507005 0.53494936]
 [0.00463683 0.00448168 0.01841735 0.08145204 0.2402795  0.65073264]] (0.957 sec)
INFO:tensorflow:loss = 0.86632836, step = 20801 (1.932 sec)
INFO:tensorflow:probabilities = [[0.00007121 0.00254582 0.01627037 0.1220782  0.3051415  0.5538929 ]
 [0.00000655 0.00118602 0.00706519 0.12233355 0.33372003 0.5356887 ]
 [0.00032805 0.00182196 0.00150136 0.04664377 0.22127838 0.72842646]
 ...
 [0.00103545 0.00123745 0.01032256 0.05181941 0.20678665 0.72879845]
 [0.00060633 0.01116641 0.03491852 0.17320304 0.31311205 0.46699366]
 [0.00216701 0.01713034 0.06022504 0.16032913 0.28291

INFO:tensorflow:global_step/sec: 57.7726
INFO:tensorflow:probabilities = [[0.00095979 0.00620226 0.02434034 0.12193943 0.2845903  0.5619678 ]
 [0.00000365 0.00092751 0.00354304 0.08914425 0.3093807  0.59700084]
 [0.0000038  0.00010372 0.00163545 0.06322883 0.29357415 0.6414541 ]
 ...
 [0.00010183 0.00500651 0.01285667 0.12695237 0.3060739  0.5490088 ]
 [0.00012173 0.00193864 0.00497302 0.06084735 0.2234248  0.70869446]
 [0.00033215 0.01067691 0.01522131 0.10564856 0.28070334 0.5874178 ]] (0.903 sec)
INFO:tensorflow:loss = 0.9469331, step = 21601 (1.734 sec)
INFO:tensorflow:probabilities = [[0.00006775 0.00174062 0.00672235 0.07353158 0.25403437 0.6639033 ]
 [0.00000153 0.00020502 0.00094066 0.04279706 0.2502715  0.7057842 ]
 [0.00000119 0.00041936 0.0079275  0.13540517 0.35511664 0.5011301 ]
 ...
 [0.00000001 0.00000072 0.00004447 0.01962112 0.2622116  0.71812207]
 [0.00084324 0.00512588 0.0088617  0.07722574 0.24186496 0.6660784 ]
 [0.00000006 0.00006665 0.00035989 0.03824874 0.261967

INFO:tensorflow:global_step/sec: 54.3476
INFO:tensorflow:probabilities = [[0.00051732 0.0245516  0.0529344  0.16530754 0.2790258  0.47766334]
 [0.00000011 0.00203598 0.00517268 0.13929068 0.4095539  0.4439467 ]
 [0.00001013 0.00081564 0.00462948 0.09029673 0.30174288 0.602505  ]
 ...
 [0.02010148 0.01177278 0.01958976 0.06688067 0.19437754 0.6872778 ]
 [0.         0.00000657 0.00127901 0.11914167 0.44597137 0.4336014 ]
 [0.00003292 0.00109161 0.00694574 0.09255777 0.29421815 0.6051538 ]] (0.891 sec)
INFO:tensorflow:loss = 0.9145263, step = 22401 (1.843 sec)
INFO:tensorflow:probabilities = [[0.00198974 0.01853134 0.01723245 0.08577342 0.22208522 0.65438783]
 [0.00021188 0.0185139  0.03474177 0.19906855 0.32717952 0.42028433]
 [0.00002461 0.00141457 0.00137021 0.04165241 0.2065712  0.748967  ]
 ...
 [0.00000018 0.0000237  0.000815   0.04031817 0.25552526 0.7033177 ]
 [0.00179517 0.01375833 0.01831877 0.1079808  0.26505467 0.5930922 ]
 [0.00001593 0.00113971 0.00439538 0.10314701 0.309832

INFO:tensorflow:global_step/sec: 53.9517
INFO:tensorflow:probabilities = [[0.00000426 0.00029261 0.00139819 0.05340225 0.26842612 0.6764766 ]
 [0.00000309 0.00039337 0.00293359 0.0834509  0.31021857 0.6030005 ]
 [0.00022886 0.00439567 0.007988   0.09128001 0.2639157  0.6321918 ]
 ...
 [0.00059554 0.03689134 0.04313995 0.17476565 0.29600883 0.44859865]
 [0.00000666 0.0012602  0.00796565 0.10590596 0.3428654  0.5419962 ]
 [0.0002834  0.00637239 0.02475844 0.11296267 0.26277697 0.5928461 ]] (0.897 sec)
INFO:tensorflow:loss = 0.9933211, step = 23201 (1.872 sec)
INFO:tensorflow:probabilities = [[0.00000042 0.00013647 0.00142716 0.09706147 0.4009833  0.5003912 ]
 [0.00000075 0.00078288 0.00277531 0.06895889 0.29175884 0.6357233 ]
 [0.00021397 0.09229833 0.00945771 0.17925005 0.41938055 0.29939944]
 ...
 [0.00041131 0.00122692 0.00280246 0.04663563 0.21747464 0.731449  ]
 [0.00028824 0.00309943 0.01277531 0.10961048 0.28635135 0.5878751 ]
 [0.00021317 0.00531299 0.01626522 0.13962506 0.307480

...

INFO:tensorflow:global_step/sec: 53.5392
INFO:tensorflow:probabilities = [[0.00005562 0.00251724 0.00518961 0.10088973 0.30271783 0.58862996]
 [0.00000136 0.00002865 0.00094542 0.06295828 0.32424918 0.61181706]
 [0.00068747 0.00039715 0.00094376 0.03166134 0.19377522 0.77253497]
 ...
 [0.0004594  0.01365294 0.02899437 0.1444008  0.31445438 0.49803802]
 [0.00002318 0.00183176 0.01114422 0.12507781 0.32753256 0.5343904 ]
 [0.00029036 0.0040903  0.01360846 0.08302186 0.25840384 0.6405851 ]] (0.914 sec)
INFO:tensorflow:loss = 0.9226439, step = 36801 (1.862 sec)
INFO:tensorflow:probabilities = [[0.00002372 0.00462843 0.00734774 0.11448493 0.3061327  0.5673826 ]
 [0.00000023 0.00071694 0.00498339 0.05945797 0.2524315  0.68241   ]
 [0.00066568 0.04245838 0.05610062 0.1509054  0.26753393 0.48233598]
 ...
 [0.00000141 0.00031356 0.00333876 0.06200903 0.395046   0.53929126]
 [0.00149314 0.00575034 0.00696183 0.06122191 0.22418605 0.7003867 ]
 [0.01003552 0.02790994 0.06635853 0.13321406 0.245492

INFO:tensorflow:global_step/sec: 52.7986
INFO:tensorflow:probabilities = [[0.00000398 0.00271239 0.00931173 0.0961475  0.31756073 0.5742637 ]
 [0.00048153 0.00805305 0.01320755 0.1516081  0.34422097 0.48242885]
 [0.00000019 0.00067661 0.00775436 0.15633518 0.394401   0.44083264]
 ...
 [0.         0.00000162 0.0000066  0.01002959 0.2116581  0.7783041 ]
 [0.0000003  0.00036656 0.00103715 0.04790315 0.27607325 0.67461956]
 [0.00002484 0.00297447 0.00509951 0.08988971 0.2882784  0.61373305]] (0.950 sec)
INFO:tensorflow:loss = 0.86219287, step = 37601 (1.894 sec)
INFO:tensorflow:probabilities = [[0.0007326  0.01281013 0.03027229 0.15513082 0.32794335 0.47311077]
 [0.00117219 0.01009188 0.01019596 0.07815148 0.24069239 0.6596961 ]
 [0.00003815 0.00545246 0.01107895 0.10884946 0.30027017 0.5743108 ]
 ...
 [0.00205673 0.0147267  0.01368023 0.07621276 0.22469632 0.66862726]
 [0.00001855 0.01849143 0.02746692 0.16727267 0.3369298  0.44982064]
 [0.00005147 0.0001272  0.00010497 0.01264591 0.13141

INFO:tensorflow:global_step/sec: 51.965
INFO:tensorflow:probabilities = [[0.00000916 0.00427125 0.0144118  0.11738185 0.31393665 0.5499893 ]
 [0.00043939 0.03458076 0.02664563 0.20612414 0.39567673 0.33653334]
 [0.00009052 0.00454379 0.00340609 0.05112796 0.22777972 0.7130519 ]
 ...
 [0.00000546 0.00083191 0.00316823 0.06196992 0.27399617 0.6600283 ]
 [0.00000096 0.00073582 0.00492175 0.10801136 0.34918323 0.53714687]
 [0.00032214 0.00686506 0.03388092 0.19416244 0.28307775 0.48169166]] (1.013 sec)
INFO:tensorflow:loss = 0.93213695, step = 38401 (1.921 sec)
INFO:tensorflow:probabilities = [[0.00001149 0.001887   0.01412755 0.13320614 0.33546466 0.5153032 ]
 [0.00008194 0.00275507 0.0144092  0.10368449 0.28900725 0.590062  ]
 [0.00000775 0.0000956  0.00039565 0.01676924 0.14725733 0.83547443]
 ...
 [0.00000343 0.00078184 0.00367571 0.06305715 0.27954128 0.65294063]
 [0.00037341 0.00159126 0.00168931 0.03136776 0.17590532 0.78907293]
 [0.00017218 0.00600522 0.01505214 0.09921667 0.278635

INFO:tensorflow:global_step/sec: 54.7236
INFO:tensorflow:probabilities = [[0.00002204 0.00132623 0.00658248 0.06889575 0.26746714 0.65570635]
 [0.00245804 0.00770717 0.00712439 0.0586017  0.20978045 0.7143282 ]
 [0.00002415 0.00538673 0.02052478 0.15100124 0.35353383 0.46952927]
 ...
 [0.00002035 0.00785381 0.0061472  0.07795008 0.26272306 0.6453055 ]
 [0.00000001 0.00003874 0.00003565 0.02171635 0.23609501 0.74211425]
 [0.0001421  0.01178906 0.01497282 0.10314125 0.2764431  0.5935117 ]] (0.929 sec)
INFO:tensorflow:loss = 0.9055774, step = 39201 (1.818 sec)
INFO:tensorflow:probabilities = [[0.00001304 0.00098987 0.00248067 0.03946521 0.20120116 0.75585   ]
 [0.00000023 0.00023135 0.00209685 0.07623906 0.32535157 0.59608096]
 [0.00000317 0.00070253 0.00201117 0.06134425 0.271773   0.66416585]
 ...
 [0.00000021 0.00006492 0.00010712 0.01707681 0.18348475 0.7992661 ]
 [0.00000174 0.00415134 0.01188025 0.16461356 0.40487576 0.41447732]
 [0.00000212 0.00004341 0.00002714 0.00609598 0.104766

INFO:tensorflow:Saving checkpoints for 40000 into ./tmp/pingfen_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.9487269.


## 评估模型

In [12]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-03-12-04:02:28
INFO:tensorflow:Restoring parameters from ./tmp/pingfen_convnet_model/model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2018-03-12-04:02:29
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.6092, global_step = 40000, loss = 0.9042214
{'loss': 0.9042214, 'global_step': 40000, 'accuracy': 0.6092}


In [13]:
pd_predict = pd.read_csv('./data/pd_predict.csv',index=False)